# Initial Recon
This notebook contains commands and procedures for conducting initial reconnaissance on a target. Customize the variables below according to your specific requirements.


## Global Variables/Commands

In [24]:
# Imports
import os
import subprocess
import datetime
from IPython.display import display, Markdown
import paramiko
import pickle

In [25]:
# Global Variables
_var_target_ip = '127.0.0.1'
_var_assess_name = 'TestAssess1'
_var_log_location = f'Logs/{_var_assess_name}'

In [26]:
# Append to Log
if not os.path.exists(_var_log_location):
    open(f'{_var_log_location}.txt', 'a').close()
    
def _func_append_to_log(source, command, output):
    with open(f'{_var_log_location}.txt', 'a') as log_file:
        timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        log_file.write(f'Timestamp: {timestamp}\n')
        log_file.write(f'Source: {source}\n')
        log_file.write(f'Command: {command}\n')
        log_file.write(f'Output:\n{output}\n')
        log_file.write('=' * 70 + '\n')


In [27]:
#VM Config
_var_vm_host = '192.168.138.130'
_var_vm_port = 22
_var_vm_username = 'bloodvault'
_var_vm_password = ''
_var_vm_key = 'Keys/kali_dev'


In [28]:
# Run VM Commands
def _command_exe_vm(command):
    # Initialize the SSH client
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    
    # Connect to the VM
    if _var_vm_key:
        key = paramiko.RSAKey.from_private_key_file(_var_vm_key)
        client.connect(hostname=_var_vm_host, port=_var_vm_port, username=_var_vm_username, pkey=key)
    else:
        client.connect(hostname=_var_vm_host, port=_var_vm_port, username=_var_vm_username, password=_var_vm_password)
    
    # Execute the command
    stdin, stdout, stderr = client.exec_command(command)
    output = stdout.read().decode('utf-8')
    error = stderr.read().decode('utf-8')
    
    # Close the SSH connection
    client.close()
    #TODO Returning errors looks like trash in output, if added in return
    #if error == '':
    #    _func_append_to_log('VM', command, output)
    #    return output
    #else:
    #    _func_append_to_log('VM', command, f'Error: {error}')
    #    return output
    _func_append_to_log('VM', command, output)
    return output

In [29]:
# Run Windows Commands Locally
def _command_exe_win(command):
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out, err = process.communicate()
    if process.returncode == 0:
        output = out.decode('utf-8').strip()
    else:
        output = err.decode('utf-8').strip()
    
    _func_append_to_log('Local', command, output)
    return output

In [30]:
# Start Caching and Command Execution
def cache_data(data, file_path):
    with open(file_path, 'wb') as file:
        pickle.dump(data, file)

def load_cached_data(file_path):
    if os.path.exists(file_path):
        with open(file_path, 'rb') as file:
            return pickle.load(file)

def cache_command_output(cache_id, force_run, os_type, run_command):
    cache_folder = 'Cache'
    unique_id = f'{cache_id}.pkl'
    cache_file_path = os.path.join(cache_folder, unique_id)

    os.makedirs(cache_folder, exist_ok=True)

    cached_data = load_cached_data(cache_file_path)
    if cached_data is not None and not force_run:
        print('[*] Loading data from cache.')
        return cached_data
    else:
        print('[*] Cache not found or execution forced, executing command.')
        if os_type == 'win':
            result = _command_exe_win(run_command)
        elif os_type == 'vm':
            result = _command_exe_vm(run_command)
        cache_data(result, cache_file_path)
        return result

### Action Template

```python
#Template for new actions

#Whether to force the command to run regardless of cache
#force_run = False

#The name of the cache file
#cmd_cache = ''

#The syntax passed to the command
#cmd_syntax = ''

#Whether to run the command on local system or VM (accepts 'win' or 'vm')
#cmd_os = ''

#The cache and run function, no edits necessary
#output = cache_command_output(cmd_cache, force_run, cmd_run)

#print(output)
```

In [31]:
# Template for new actions

#force_run = False

#cmd_cache = ''
#cmd_syntax = ''
#cmd_os = ''

#output = cache_command_output(cmd_cache, force_run, cmd_run)

#print(output)

## Environment Check


In [32]:
# Find External IP
force_run = True

cmd_cache = 'ext_ip'
cmd_os = 'vm'
cmd_syntax = 'curl ip.beer'

ext_ip = cache_command_output(cmd_cache, force_run, cmd_os, cmd_syntax).strip()


print(ext_ip)

[*] Cache not found or execution forced, executing command.


72.193.145.223


## Scope

In [33]:
# Alives Check
force_run = False

cmd_cache = f'ping_{_var_target_ip}'
cmd_syntax = f'ping -n 4 {_var_target_ip}' 
cmd_os = 'win'

ping = cache_command_output(cmd_cache, force_run, cmd_os, cmd_syntax)

print(ping)

[*] Loading data from cache.
Pinging 127.0.0.1 with 32 bytes of data:
Reply from 127.0.0.1: bytes=32 time<1ms TTL=128
Reply from 127.0.0.1: bytes=32 time<1ms TTL=128
Reply from 127.0.0.1: bytes=32 time<1ms TTL=128
Reply from 127.0.0.1: bytes=32 time<1ms TTL=128

Ping statistics for 127.0.0.1:
    Packets: Sent = 4, Received = 4, Lost = 0 (0% loss),
Approximate round trip times in milli-seconds:
    Minimum = 0ms, Maximum = 0ms, Average = 0ms


In [34]:
# DNS Lookup
force_run = True
cmd_cache = f'nslookup_{ext_ip}'
cmd_syntax = f'nslookup {ext_ip}'
cmd_os = 'win'

dns = cache_command_output(cmd_cache, force_run, cmd_os, cmd_syntax)

print(dns)

[*] Cache not found or execution forced, executing command.
Server:  dns.google
Address:  8.8.8.8

Name:    ip72-193-145-223.lv.lv.cox.net
Address:  72.193.145.223


In [35]:
# Hostname from VM
force_run = False
cmd_cache = 'vmhostname'
cmd_syntax = f'hostname'
cmd_os = 'vm'

_var_vm_hostname = cache_command_output(cmd_cache, force_run, cmd_os, cmd_syntax)

print(_var_vm_hostname)

[*] Loading data from cache.
kalidev

